<a href="https://colab.research.google.com/github/ORAzzQWQ/NLP_2024/blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==2.4.0
!pip install transformers==4.37.0
!pip install datasets==2.21.0
!pip install accelerate==0.21.0
!pip install scikit-learn==1.5.2
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 814.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import transformers as T
from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm import tqdm
from torchmetrics import SpearmanCorrCoef, Accuracy, F1Score
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# 有些中文的標點符號在tokenizer編碼以後會變成[UNK]，所以將其換成英文標點
token_replacement = [
    ["：" , ":"],
    ["，" , ","],
    ["“" , "\""],
    ["”" , "\""],
    ["？" , "?"],
    ["……" , "..."],
    ["！" , "!"]
]

In [4]:
class SemevalDataset(Dataset):
    def __init__(self, split="train") -> None:
        super().__init__()
        assert split in ["train", "validation"]
        self.data = load_dataset(
            "sem_eval_2014_task_1", split=split, cache_dir="./cache/"
        ).to_list()

    def __getitem__(self, index):
        d = self.data[index]
        # 把中文標點替換掉
        for k in ["premise", "hypothesis"]:
            for tok in token_replacement:
                d[k] = d[k].replace(tok[0], tok[1])
        return d

    def __len__(self):
        return len(self.data)

data_sample = SemevalDataset(split="train").data[:3]
print(f"Dataset example: \n{data_sample[0]} \n{data_sample[1]} \n{data_sample[2]}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for sem_eval_2014_task_1 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sem_eval_2014_task_1.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/4500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4927 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset example: 
{'sentence_pair_id': 1, 'premise': 'A group of kids is playing in a yard and an old man is standing in the background', 'hypothesis': 'A group of boys in a yard is playing and a man is standing in the background', 'relatedness_score': 4.5, 'entailment_judgment': 0} 
{'sentence_pair_id': 2, 'premise': 'A group of children is playing in the house and there is no man standing in the background', 'hypothesis': 'A group of kids is playing in a yard and an old man is standing in the background', 'relatedness_score': 3.200000047683716, 'entailment_judgment': 0} 
{'sentence_pair_id': 3, 'premise': 'The young boys are playing outdoors and the man is smiling nearby', 'hypothesis': 'The kids are playing outdoors near a man with a smile', 'relatedness_score': 4.699999809265137, 'entailment_judgment': 1}


In [5]:
train_dataset = SemevalDataset(split="train")
validation_dataset = SemevalDataset(split="validation")

tokenizer = T.BertTokenizer.from_pretrained("google-bert/bert-base-uncased", cache_dir="./cache/")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
# Define the hyperparameters
lr = 3e-5
epochs = 3
train_batch_size = 8
validation_batch_size = 8

In [7]:
# TODO1: Create batched data for DataLoader
# `collate_fn` is a function that defines how the data batch should be packed.
# This function will be called in the DataLoader to pack the data batch.
# ASK WITH Claude

def collate_fn(batch):
    # TODO1-1: Implement the collate_fn function
    # Write your code here
    # The input parameter is a data batch (tuple), and this function packs it into tensors.
    premises = [item['premise'] for item in batch]
    hypotheses = [item['hypothesis'] for item in batch]
    entailment_labels = torch.tensor([item['entailment_judgment'] for item in batch])
    relatedness_score = torch.tensor([item['relatedness_score'] for item in batch])

    # Use tokenizer to pack tokenize and pack the data and its corresponding labels.
    encoded = tokenizer(
        premises,
        hypotheses,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

    # Return the data batch and labels for each sub-task.
    return {
        'input_ids': encoded['input_ids'],
        'attention_mask': encoded['attention_mask'], # tokenizer自動生成的mask [pad]的東東
        'entailment_labels': entailment_labels,
        'relatedness_score': relatedness_score
    }

# TODO1-2: Define your DataLoader
dl_train = DataLoader(
    train_dataset,
    batch_size=train_batch_size,
    shuffle=True,
    collate_fn=collate_fn
)
dl_validation = DataLoader(
    validation_dataset,
    batch_size=validation_batch_size,
    shuffle=False,
    collate_fn=collate_fn
)

In [11]:
# TODO2: Construct your model
# ASK WITH Claude

class MultiLabelModel(torch.nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Write your code here
        # Define what modules you will use in the model
        self.bert = T.BertModel.from_pretrained('bert-base-uncased')
        self.dropout = torch.nn.Dropout(0.1)

        # 1. 相似度分數 (回歸任務)
        self.similarity_head = torch.nn.Linear(768, 1)
        # 2. 蘊含判斷 (分類任務)
        self.entailment_head = torch.nn.Linear(768, 3)

    def forward(self, input_ids, attention_mask): # 明確定義需要的參數,不然會報錯(ASK WITH Claude)
        # Write your code here
        # Forward pass
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)

        # 產生預測
        similarity_scores = self.similarity_head(pooled_output).squeeze(-1)
        entailment_logits = self.entailment_head(pooled_output)

        return similarity_scores, entailment_logits

In [14]:
# ASK WITH Claude
model = MultiLabelModel().to(device)
# TODO3: Define your optimizer and loss function

# TODO3-1: Define your Optimizer
optimizer = AdamW(model.parameters(), lr=lr)

# TODO3-2: Define your loss functions (you should have two)
# Write your code here
similarity_loss_fn = torch.nn.MSELoss()  # 回歸任務用MSE
entailment_loss_fn = torch.nn.CrossEntropyLoss()  # 分類任務用CrossEntropy

# scoring functions
spc = SpearmanCorrCoef().to(device)
acc = Accuracy(task='multiclass', num_classes=3).to(device)  # 因為entailment是3分類任務
f1 = F1Score(task='multiclass', num_classes=3).to(device)

In [17]:
# ASK WITH Claude

for ep in range(epochs):
    pbar = tqdm(dl_train)
    pbar.set_description(f"Training epoch [{ep+1}/{epochs}]")
    model.train()
    # TODO4: Write the training loop
    # Write your code here
    # train your model

    for batch in pbar:
        # 將數據移到設備上
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        relatedness_score = batch['relatedness_score'].float().to(device)
        entailment_labels = batch['entailment_labels'].to(device)

        # clear gradient
        optimizer.zero_grad()

        # forward pass
        similarity_preds, entailment_logits = model(input_ids=input_ids, attention_mask=attention_mask)

        # compute loss
        similarity_loss = similarity_loss_fn(similarity_preds, relatedness_score)
        entailment_loss = entailment_loss_fn(entailment_logits, entailment_labels)
        total_loss = similarity_loss + entailment_loss

        # back-propagation
        total_loss.backward()

        # model optimization
        optimizer.step()

        # 更新進度條
        pbar.set_postfix(loss=total_loss.item())

    pbar = tqdm(dl_validation)
    pbar.set_description(f"Validation epoch [{ep+1}/{epochs}]")
    model.eval()
    # TODO5: Write the evaluation loop
    # Write your code here
    # Evaluate your model
    similarity_preds_list = []
    similarity_labels_list = []
    entailment_preds_list = []
    entailment_labels_list = []

    with torch.no_grad():
        for batch in pbar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            similarity_labels = batch['relatedness_score'].float().to(device)
            entailment_labels = batch['entailment_labels'].to(device)

            similarity_preds, entailment_logits = model(input_ids, attention_mask)

            similarity_preds_list.append(similarity_preds.cpu())
            similarity_labels_list.append(similarity_labels.cpu())

            entailment_preds = torch.argmax(entailment_logits, dim=1)
            entailment_preds_list.append(entailment_preds.cpu())
            entailment_labels_list.append(entailment_labels.cpu())
    # Output all the evaluation scores (SpearmanCorrCoef, Accuracy, F1Score)
    all_similarity_preds = torch.cat(similarity_preds_list)
    all_similarity_labels = torch.cat(similarity_labels_list)
    spearman = spc(all_similarity_preds, all_similarity_labels)

    all_entailment_preds = torch.cat(entailment_preds_list)
    all_entailment_labels = torch.cat(entailment_labels_list)
    accuracy = acc(all_entailment_preds, all_entailment_labels)
    f1_score = f1(all_entailment_preds, all_entailment_labels)

    print(f"Validation Metrics:")
    print(f"- Spearman Correlation: {spearman:.4f}")
    print(f"- Accuracy: {accuracy:.4f}")
    print(f"- F1 Score: {f1_score:.4f}")

    torch.save(model, f'./ep{ep}.ckpt')

Validation epoch [1/3]: 100%|██████████| 63/63 [00:01<00:00, 50.54it/s]


Validation Metrics:
- Spearman Correlation: 0.8295
- Accuracy: 0.8620
- F1 Score: 0.8620


Validation epoch [2/3]: 100%|██████████| 63/63 [00:01<00:00, 50.14it/s]


Validation Metrics:
- Spearman Correlation: 0.8391
- Accuracy: 0.8580
- F1 Score: 0.8580


Validation epoch [3/3]: 100%|██████████| 63/63 [00:01<00:00, 49.87it/s]


Validation Metrics:
- Spearman Correlation: 0.8220
- Accuracy: 0.8840
- F1 Score: 0.8840


For test set predictions, you can write perform evaluation simlar to #TODO5.